#### Lecture 1. Self-Supervised Pre_training

Recent Trends
- transformer model 및 self-attention model 이 일반적 목적의 sequence encoder & decoder의 대세가 되는 추세
- transformer에서 model 변경없이 self-attention block을 deep 하게 stack하여 좋은 성능
- 추천 시스템, 신약 개발, computer vision 에도 영역 확장
- self-attention 기반의 모델은 greedy decoding이라는 한계는 지님

GPT-1 (Generative Pre-Training  by open AI)
- special token 도입하여 여러 task를 동시에 커버가능한 통합된 모델 제안
- input과 position embedded vector에 self-attention block 12개를 통과시킴
- 순차적으로 다음 단어를 예측하는 language model에 의해 GPT-1 모델 학습
- < EOS > token을 Extract token으로 변경하여 최종 output layer의 input으로 주어 문장의 긍정/부정 분석
- 문단이 다수의 문장으로 이루어진 경우 하나의 sentence로 취급하고 원래의 문장 사이에 Delim token 을 추가하여 문장관의 관계(내포, 모순)를 예측
- 문장간 유사도 및 다중 선택 기능도 추가됨
- task 분류를 위해서 추가적인 한 layer를 추가하여 pre-training된 layer는 학습률을 낮추고 fine-tuning 진행
- pre-training 당시의 data는 별도의 label이 요구되지 않고 대규모 데이터로 다음 단어를 예측하는 간단한 모델으로 학습이 이루어짐
- main task인 문서 분류의 경우 data에 label이 요구되는데 label datat는 한정적이므로 self-supervised learning 을 통해 대규모의 데이터의 지식을 전이 학습
- task만을 위해 customized 된 모델과 test에만 대응하는 소량의 data로 학습한 경우보다 더 좋은 성능을 보임

BERT
- 현재까지도 가장 널리 쓰이는 pre-training model
- LSTM 기반의 pre-trained model도 존재 (ELMo : Bi-LSTM)
- Masked Language Model (MLM) : 추측하려는 단어의 앞뒤 모두 파악 후 추론
    - input token의 일부(평균 15%)를 가린 후 추론
    - 너무 적으면 : 학습의 비효율성, 너무 많으면 : 정보 추출 불가
    - 전부다 < MASK > 로 치환 시 데이터  편향된 모델 생성 가능성
        - 80%  < MASK > , 10%  random word, 10% 는 원래 데이터 유지
- Pre-training tasks in Bert : Next sentence prediction
    - 주어진 두 문장이 서로 연결된 문장인지 판단하는 label 부여
    - < CLS > : 다수의 문장레벨에서의 예측 토큰으로 문장의 앞에 추가
- model architecture
    - BERT base : self attention layer L=12, encoding vector dim H=768, attention head A=12
    - BERT large: L=12, H=1024, A=16
- input representation
    - world piece embedding : 각각의 sub word로 embedding, 30,000개의 wordpiece, 의미단위를 담을 수있는 보다 진보적 형태
    - positional vector 도 학습에 의해 결정
    - 문장 단위의 인덱스를 반영한 segment embedding 추가
- BERT 와 GPT의 차이점
    - GPT : 바로 다음 단어를 예측하는 task를 수행해야 하기 때문에 다음 단어의 접근이 불가능( Unidirectional )
    - BERT : Mask 단어를 포함하여 모든 단어 접근이 가능 (self - attention module 사용)
    - 학습 데이터  GPT : Book corpus(800M words) // BERT : Wikipedia(2,500M words)
    - Batch size  GPT : 32,000 words  BERT : 128,000 words
    - 일반적으로 batch size 의 크기가 클수록 더 많은 표본에 대한 gradient 계산값의 평균을 구하므로 학습이 안정적이나 메모리가 이에 비례하여 증가하므로 한계는 존재
- BERT 의 모델을 기울수록 끊임없이 성능이 좋아짐 (110M -> 340M parameter)
    
BERT 로 수행할 수 있는 Task
- Machine Reading Comprehension(MRC) : 주어진 지문을 이해하고 질문에 답하기
    - SQuAD(Standford Question Answering Dataset) 로 학습
    - 각각의 word encoding vector를 FC 시킨 scalar 값을 구하고 답에 해당하는 문구가 어디에서부터 시작하는 지를 예측
    - FC의 가중치값이 fine tuning을 통해 Random initialization parameter로부터 학습됨
    - starting point 및 ending point를 위한 FC layer를 각각 학습
    - 질문에 대한 답이 문단내에 없을 경우 CLS token을 no answer에 해당하는 logit으로 삼음
- SWAG : 다음에 올 문장 예측하기
    - 주어진 문장와 예시문 각각을 concat 하여 CLS token들을 FC 시킨 scalar 생성 하고 각각의 scalar에 대해 softmax를 수행한뒤 ground truth와 비교하여 학습시킴

#### Lecture 2. Advanced Self-supervised Pre-training Models

GPT-2
- model 구조는 GPT-1과 유사하며 크기가 커진 transformer language model
- 40 GB 분량 text 로 훈련(reddit 이라는 web community에 추천을 받은 링크 위주)
- 긴 글을 sequence처럼 생성하며 유려한 문장 구사
- 문장의 긍정/부정, 번역, 독해 같은 모든 NLP의 task가 질의 응답으로 환원할 수 있다는 발상을 모티브
- BERT 의 word piece와 유사하게 BPE(Byte Pair Encoding) 사용
- layer가 뒤로 갈수록 여러 선형 변환 값이 0에 가까워지도록 layer weight 조정
- fine - tuning 과정 없이 zero - shot setting 사용

GPT-3
- GPT-2 에 비해 훨씬 많은 수의 parameter 사용
- 주어진 task와 하나의 예시(one shot), 몇개의 예시(few shot)를 주었을 때 좋은 성능
- few example로 부터 동적으로 답을 생성하며, model 이 클수록 적응력이 뛰어남

ALBERT : A Lite BERT
- 경량화 버전의 BERT 성능이 별로 하락하지 않도록 dimension size 유지
- word embedding layer V X H matrix 를 (V X E) x (E X H) matrix 로 쪼개어 parameter 수 감소(row-rank matrix factorization)
- self-attention block 별로 존재하는 matrix의 parameter들을 공유함
- feed-forward network parameter 와 attention parameter를 모두 공유할때 parameter 숫자는 비약적으로 감소하지만 성능은 별 차이가 없음
- sentence order prediction(SOP) : BERT의 next sentence prediction(NSP)이 의미가 없음
- 동일한 문서에서 가져온 연속된 두개의 segment(문장)을 정순서/역순서로 배치하여 모델을 통해 순서가 제대로 된 순서인지 반대로 된 것인지 예측하도록 함

ELECTRA : Efficiently Learning an Encoder that Classifies Token Replacements Accurately
- Generator(Language Modeling을 통해 단어를 복원해주는 것) - BERT model
- Discriminator : transformer의 self-attention block으로 단어별로 이진분류(original / replaced)하는 역할
- GAN model에서 motive를 따옴
- discriminator에 pre-training 수행

Light-weight Models : 비대해진 model을 적은 layer나 parameter로 압축한 모델
- DistillBERT : teacher model / student model
    - student model은 teacher model의 경량화 버전으로 ground truth 를 teacher model의 
      softmax output distribution으로 사용함으로써 teacher model의 성능 모사
- TinyBERT
    - 위의 teacher/student model에 덧붙여 각 block의 attention matrix, hidden state
       vector 들까지도 모두 student model이 모사하도록 설계

Fusing knowledge graph into language model
- BERT는 문맥, 단어간 유사도는 잘 파악하지만 문장에 나타나지 않은 추가적인 정보를 활용하지는 못함, common sense 부족
- 외부지식들이 knowledge graph로 표현되므로 이를 접목시키려는 시도